# Calculate and save the transformation matrices for each time point in RAS space

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from numpy.linalg import svd
import numpy as np
import math
from scipy import interpolate
import random
from numpy.linalg import inv
import scipy.io
import pydicom
import cmath
from scipy.spatial.transform import Rotation as R
import os
import datetime
import csv
import datetime

In [ ]:
from utils import *

In [ ]:
# get all subjects with GA:
GA = []
for root, dirs, files in os.walk("./GA/extracted"):
    for d in dirs:
        GA.append(d.split(" ")[-1])
# get all subjects without GA:
no_GA = []
for root, dirs, files in os.walk("./No GA/extracted"):
    for d in dirs:
        no_GA.append(d.split(" ")[-1])
subjs = np.append(no_GA, GA)
print(subjs)

In [ ]:
# define scan times for each sequence:
sequ = ['Head_t1_mprage_sag_p2_iso', 'Head_t1_tirm_tra_darkfluid_p2', 'Head_t2_tirm_tra_darkfluid_p2', 'Head_t2_tirm_cor_darkfluid_4mm', 'Head_t2_blade_tra_5mm_GD', 'Head_t1_mprage_sag_p2_iso_GD']
sequ_abbr = ['t1_mprage_sag', 't1_tirm_tra', 't2_tirm_tra', 't2_tirm_cor', 't2_blade_tra_5mm_GD', 't1_mprage_sag*GD']
scantimes = [266, 174, 272, 164, 122, 266]   # in seconds from TCL_mMR_documentation_BUF.xlsx

In [ ]:
# get scan start for each sequ type and subject:
save = ['Subj_ID, sequence, start_time, end_time']
for sub in subjs:
    for seq, sct in zip(sequ_abbr, scantimes):
        path = '/mnt/mocodata1/BUF/DataMMR/'+sub+'/PETMR*/*'+seq+'*/*'
        find = glob.glob(path)
        if seq == 't1_mprage_sag':
            if len(find)>1:
                find = [s for s in find if "GD" not in s]
        if len(find) > 0:
            file = find[0]
            start_time = getAcqTimeFromDICOM(file)
            # divide sct in minutes and seconds:
            mins = sct//60
            scs = sct%60
            start_tim = datetime.datetime.strptime(start_time, '%H%M%S.%f')
            timeToAdd = datetime.timedelta(minutes=mins, seconds=scs)
            end_time = (start_tim+ timeToAdd).strftime('%H%M%S.%f')
            save.append(sub+', '+seq+', '+str(start_time)+', '+str(end_time))
save = np.array(save)
np.savetxt('./new_calc/Scan_Times.txt', save, fmt="%s")

In [ ]:
# load scan start for each sequence:
Scan_Times = np.loadtxt('./new_calc/Scan_Times.txt', dtype=str, delimiter=', ', skiprows=1)

In [ ]:
#get list of subjects
subjs = []
for root, dirs, files in os.walk("./regions binarization"): 
    for d in dirs:
        subjs.append(d)

In [ ]:
print(subjs)

Calculate matrices with reference position in middle of MPRAGE:

In [ ]:
# file with dicom info (id,age,scan start)
file = "./dicomData.csv"
path = "./subjects data/"
A_dcs2ecs=np.array([[-1, 0, 0, 0], [0, 1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]])

for d in no_GA:
    aln, tim, poa, log  = getFileNames(path+d)
    print(aln,tim,poa,log)
    A_tcs2dcs = readtcs2dcs(aln)
    A_tcs2ecs = A_dcs2ecs@A_tcs2dcs
    time = getTime(file,d)
    print(time,d)
    # add half of MPRAGE acquisition time (266s)
    timeToAdd = datetime.timedelta(minutes=2, seconds=13)
    time_ = datetime.datetime.strptime(time, '%H%M%S.%f')
    time = (time_+ timeToAdd).strftime('%H%M%S.%f')
    refFrame = 0
    # get the ref frame (start of scan time) either from tim or log file
    if tim!="":
        print("IN TIM")
        timeDifference = getTimeDifference(log)
        print(timeDifference)
        timeDiffList = timeDifference.split(":")
        a = timeDiffList[-1].split(".")
        # convert to python timedate
        timeToAdd = datetime.timedelta(minutes=int(timeDiffList[1]), seconds=int(a[0]), milliseconds=int(a[1]))
        refFrame = findFrameNumber(tim, time, timeToAdd)
        print(refFrame)
    elif log!="":
        print("IN LOG")
        print(time)
        refFrame = findFrameNumberInLog(log, time)
    else:
        break
    if not os.path.exists("./new_calc/No GA/matricesREF "+d):
        os.mkdir("./new_calc/No GA/matricesREF "+d)
    refTr = transform(poa, A_tcs2ecs, refFrame, "./new_calc/No GA/matricesREF "+d)
    print(time,refFrame,refTr)

In [ ]:
# file with dicom info (id,age,scan start)
file = "./dicomData.csv"
path = "./subjects data/"
A_dcs2ecs=np.array([[-1, 0, 0, 0], [0, 1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]])

for d in GA:
    aln, tim, poa, log  = getFileNames(path+d)
    print(aln,tim,poa,log)
    A_tcs2dcs = readtcs2dcs(aln)
    A_tcs2ecs = A_dcs2ecs@A_tcs2dcs
    #time = getAcqTimeFromDICOM(dcm)
    time = getTime(file,d)
    print(time,d)
    # add half of MPRAGE acquisition time (266s)
    timeToAdd = datetime.timedelta(minutes=2, seconds=13)
    time_ = datetime.datetime.strptime(time, '%H%M%S.%f')
    time = (time_+ timeToAdd).strftime('%H%M%S.%f')
    refFrame = 0
    # get the ref frame (start of scan time) either from tim or log file
    if tim!="":
        print("IN TIM")
        timeDifference = getTimeDifference(log)
        print(timeDifference)
        timeDiffList = timeDifference.split(":")
        a = timeDiffList[-1].split(".")
        # convert to python timedate
        timeToAdd = datetime.timedelta(minutes=int(timeDiffList[1]), seconds=int(a[0]), milliseconds=int(a[1]))
        refFrame = findFrameNumber(tim, time, timeToAdd)
        print(refFrame)
    elif log!="":
        print("IN LOG")
        print(time)
        refFrame = findFrameNumberInLog(log, time)
    else:
        break
    if not os.path.exists("./new_calc/GA/matricesREF "+d):
        os.mkdir("./new_calc/GA/matricesREF "+d)
    refTr = transform(poa, A_tcs2ecs, refFrame, "./new_calc/GA/matricesREF "+d)
    print(time,refFrame,refTr)

# Analysis for excluded children:

In [ ]:
# get all excluded subjects:
subjs_in = []
for root, dirs, files in os.walk("./regions binarization"):
    for d in dirs:
        subjs_in.append(d)
subjs_all = os.listdir('./subjects data')
subjs = [s for s in subjs_all if s not in subjs_in]
print(len(subjs_in), len(subjs_all))
print(subjs)

In [ ]:
# go through GA list and sort subjs:
GA_list = ['2015-04-14_NJ', '2015-04-24_NJ',  '2015-05-05_SB',  '2016-07-08_JK',  '2016-10-19_BA',  '2017-02-24_HGSF', '2017-03-03_BA', '2017-04-21_ATWJ', '2017-05-05_AJ', '2017-09-01_MK', '2015-05-08_ADS']
noGA_list = ['2015-11-10_BSA', '2017-11-28_ToM', '2015-05-26_PN', '2015-11-03_ELV', '2015-11-13_PL', '2016-02-26_FLTH', '2016-03-11_MBH',  '2016-04-01_NLL', '2016-05-03_MBH',  '2017-01-24_JD', '2017-05-19_FSH', '2017-08-22_SB', '2017-08-29_SEKB', '2017-10-10_SB', '2016-05-03_HF', '2016-08-02_JJ', '2017-11-03_VWN']


In [ ]:
# get scan start for each sequ type and subject:
save = ['Subj_ID, sequence, start_time, end_time']
for sub in subjs:
    for seq, sct in zip(sequ_abbr, scantimes):
        path = '/mnt/mocodata1/BUF/DataMMR/'+sub+'/PETMR*/*'+seq+'*/*'
        find = glob.glob(path)
        if seq == 't1_mprage_sag':
            if len(find)>1:
                find = [s for s in find if "GD" not in s]
        if len(find) > 0:
            file = find[0]
            start_time = getAcqTimeFromDICOM(file)
            # divide sct in minutes and seconds:
            mins = sct//60
            scs = sct%60
            start_tim = datetime.datetime.strptime(start_time, '%H%M%S.%f')
            timeToAdd = datetime.timedelta(minutes=mins, seconds=scs)
            end_time = (start_tim+ timeToAdd).strftime('%H%M%S.%f')
            save.append(sub+', '+seq+', '+str(start_time)+', '+str(end_time))
save = np.array(save)
np.savetxt('./new_calc/Scan_Times_Excl.txt', save, fmt="%s")

In [ ]:
# load start time for each scan:
Scan_Times = np.loadtxt('./new_calc/Scan_Times_Excl.txt', dtype=str, delimiter=', ', skiprows=1)

In [ ]:
# file with dicom info (id,age,scan start)
file = "./dicomData.csv"
path = "./subjects data/"
A_dcs2ecs=np.array([[-1, 0, 0, 0], [0, 1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]])

for d in noGA_list:
    aln, tim, poa, log  = getFileNames(path+d)
    print(aln,tim,poa,log)
    A_tcs2dcs = readtcs2dcs(aln)
    A_tcs2ecs = A_dcs2ecs@A_tcs2dcs
    time = getTime(file,d)
    print(time,d)
    # add half of MPRAGE acquisition time (266s)
    timeToAdd = datetime.timedelta(minutes=2, seconds=13)
    time_ = datetime.datetime.strptime(time, '%H%M%S.%f')
    time = (time_+ timeToAdd).strftime('%H%M%S.%f')
    refFrame = 0
    # get the ref frame (start of scan time) either from tim or log file
    if tim!="":
        print("IN TIM")
        timeDifference = getTimeDifference(log)
        print(timeDifference)
        timeDiffList = timeDifference.split(":")
        a = timeDiffList[-1].split(".")
        # convert to python timedate
        timeToAdd = datetime.timedelta(minutes=int(timeDiffList[1]), seconds=int(a[0]), milliseconds=int(a[1]))
        refFrame = findFrameNumber(tim, time, timeToAdd)
    elif log!="":
        print("IN LOG")
        print(time)
        refFrame = findFrameNumberInLog(log, time)
    else:
        break
    if not os.path.exists("./new_calc/No GA_Excl/matricesREF "+d):
        os.mkdir("./new_calc/No GA_Excl/matricesREF "+d)
    refTr = transform(poa, A_tcs2ecs, refFrame, "./new_calc/No GA_Excl/matricesREF "+d)
    if np.sum(refTr)==0:
        print(refFrame, ' add 1')
        refFrame = str(int(refFrame)+1)
        refTr = transform(poa, A_tcs2ecs, refFrame, "./new_calc/No GA_Excl/matricesREF "+d)
    print(time,refFrame,refTr)

In [ ]:
# file with dicom info (id,age,scan start)
file = "./dicomData.csv"
path = "./subjects data/"
A_dcs2ecs=np.array([[-1, 0, 0, 0], [0, 1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]])

for d in GA_list:
    aln, tim, poa, log  = getFileNames(path+d)
    print(aln,tim,poa,log)
    A_tcs2dcs = readtcs2dcs(aln)
    A_tcs2ecs = A_dcs2ecs@A_tcs2dcs
    time = getTime(file,d)
    print(time,d)
    # add half of MPRAGE acquisition time (266s)
    timeToAdd = datetime.timedelta(minutes=2, seconds=13)
    time_ = datetime.datetime.strptime(time, '%H%M%S.%f')
    time = (time_+ timeToAdd).strftime('%H%M%S.%f')
    refFrame = 0
    # get the ref frame (start of scan time) either from tim or log file
    if tim!="":
        print("IN TIM")
        timeDifference = getTimeDifference(log)
        print(timeDifference)
        timeDiffList = timeDifference.split(":")
        a = timeDiffList[-1].split(".")
        # convert to python timedate
        timeToAdd = datetime.timedelta(minutes=int(timeDiffList[1]), seconds=int(a[0]), milliseconds=int(a[1]))
        refFrame = findFrameNumber(tim, time, timeToAdd)
        print(refFrame)
    elif log!="":
        print("IN LOG")
        print(time)
        refFrame = findFrameNumberInLog(log, time)
    else:
        break
    if not os.path.exists("./new_calc/GA_Excl/matricesREF "+d):
        os.mkdir("./new_calc/GA_Excl/matricesREF "+d)
    refTr = transform(poa, A_tcs2ecs, refFrame, "./new_calc/GA_Excl/matricesREF "+d)
    print(time,refFrame,refTr)